In [1]:
import os
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import gensim.utils
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import itertools
import pandas as pd

# filter out warnings
import warnings

[nltk_data] Downloading package punkt to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
stop = stopwords.words("english")

[nltk_data] Downloading package stopwords to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Helper method
#function to normalize description tokenize remove stopwords
def norm(desc):
    desc = str(desc)
    desc = re.sub('[^a-zA-Z]',' ',desc)
    desc = desc.lower()
    desc = re.sub(pattern='\s+', repl=' ', string=desc)
    desc = nltk.word_tokenize(desc)
    return desc


# Get  the nouns / proper nouns / adjectives
def postagwords(tagged_stanzas):
  chunkGram = r"""NP: {<NN>+<PUNCT>+}"""
  chunkGram = r"""chunk: {<JJ>+}"""
  chunkGram = r"""chunk: {<NN>+<NN>+<NN>+}"""
  #Chunk 3: Sequence of Nouns
  chunkGram = r"""chunk: {<NN|NNP|NNS|NNPS>+}"""
  chunkGram = r"""NP: {<NN|NNP|NNS|NNPS>+}"""
  chunkParser = nltk.RegexpParser(chunkGram)
  nouns = []
  finalpos= []

  for i in tagged_stanzas:
    tree1 = chunkParser.parse(i)
    nouns = [(word) for (word, pos) in i if pos == 'NN' or pos == 'NNS' or pos=='NNP' or pos=='JJ']
    finalpos.append(nouns)

  flat_list = list(itertools.chain(*finalpos))
  return flat_list



#Skills Frequency
def most_frequent_skills(flat_list):
  tag_fd = nltk.FreqDist(flat_list)
  countsskills = pd.DataFrame(tag_fd.items(), columns=['word', 'frequency'])
  # print('before')
  # print(countsskills)
  countsskills =countsskills[countsskills['word'].isin(['statistics', 'machine',  'r', 'python', 'nlp',  'neural','forecasting','tableau','power','bi',
                        'software','programming'  'cloud' , 'aws','sas','regression','predictive', 'modeling','clustering' ,'database','databases',
                        'datawarehouse','excel','pytorch','gcp','ETL','business','visualization','macros','operations','documentation','azure','ops','microsoft','powerpoint'
                        'communication','interpersonal','storytelling','marketing','terradata','splunk','ruby','spark','cassandra','mongodb','scala','api','segmentation','targeting',
                        'reporting','quality','a/b','html','css','sales','visio','logistics'
                        'project','access','qa','quality','agile','scrum','jira'])]
  countsskills.sort_values(by=['frequency'],ascending=False)
  # print('after')
  # print(countsskills)
  return countsskills[:30]

In [4]:
job_df=pd.read_csv('../data/job_postings.csv')
print(len(job_df))

15886


In [5]:
# preview
job_df.head()

,title,description,formatted_work_type,location,views,work_type
0,Licensed Insurance Agent,While many industries were hurt by the last fe...,Full-time,"Chico, CA",5.0,FULL_TIME
1,Sales Manager,Are you a dynamic and creative marketing profe...,Full-time,"Santa Clarita, CA",NaN,FULL_TIME
2,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,Contract,"New York, NY",17.0,CONTRACT
3,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,Full-time,"Forney, TX",NaN,FULL_TIME
4,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\r\nKen Fu...,Full-time,"New York, NY",2.0,FULL_TIME


In [6]:
# keep title and description column only
job_df = job_df[['title', 'description', 'views', 'work_type', 'location']]

In [7]:
#running function on description column
job_df_Desc=job_df['description'].apply(lambda x:norm(x))
job_df_Desc

0        [while, many, industries, were, hurt, by, the,...
1        [are, you, a, dynamic, and, creative, marketin...
2        [join, us, as, a, model, risk, auditor, showca...
3        [business, managerfirst, baptist, church, forn...
4        [you, could, be, one, of, the, magic, makers, ...
                               ...                        
15881    [location, west, columbia, sc, us, job, type, ...
15882    [job, title, unit, secretary, department, nurs...
15883    [job, title, radiology, aide, perdiem, departm...
15884    [grade, job, type, officer, of, administration...
15885    [nexion, health, management, affiliates, opera...
Name: description, Length: 15886, dtype: object

In [8]:
# for empty views cell replace with 0
job_df['views'].fillna(0, inplace=True)

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Khor Kean
[nltk_data]     Teng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
#removing stopwords from description
stop= nltk.corpus.stopwords.words('english')
new=['job','note','sexual','orientation','equal','opportunity','gender','plus','years','scientist','analyst','based','data','knowledge','discrimination','experience','description','summary','information','etc','eg','layers']
stop.extend(new)
job_df_clean=job_df_Desc.apply(lambda x:[item for item in x if item not in set(stop)])

In [11]:
job_df.loc[:,'tokenized_Description'] = job_df_clean
job_df.loc[:,'tokenized_Description']

0        [many, industries, hurt, last, people, still, ...
1        [dynamic, creative, marketing, professional, l...
2        [join, us, model, risk, auditor, showcase, fin...
3        [business, managerfirst, baptist, church, forn...
4        [could, one, magic, makers, ken, fulk, inc, se...
                               ...                        
15881    [location, west, columbia, sc, us, type, baker...
15882    [title, unit, secretary, department, nursing, ...
15883    [title, radiology, aide, perdiem, department, ...
15884    [grade, type, officer, administrationbargainin...
15885    [nexion, health, management, affiliates, opera...
Name: tokenized_Description, Length: 15886, dtype: object

In [12]:
df = job_df
df["description"] = df["description"].fillna("")
df["tokenized_Description"]

0        [many, industries, hurt, last, people, still, ...
1        [dynamic, creative, marketing, professional, l...
2        [join, us, model, risk, auditor, showcase, fin...
3        [business, managerfirst, baptist, church, forn...
4        [could, one, magic, makers, ken, fulk, inc, se...
                               ...                        
15881    [location, west, columbia, sc, us, type, baker...
15882    [title, unit, secretary, department, nursing, ...
15883    [title, radiology, aide, perdiem, department, ...
15884    [grade, type, officer, administrationbargainin...
15885    [nexion, health, management, affiliates, opera...
Name: tokenized_Description, Length: 15886, dtype: object

In [13]:
# save the data for faster loading
df.to_csv('../data/job_posting_clean.csv', index=False)

In [13]:
import regex as re
df['tokenized_desc_join'] = [' '.join(map(str, l)) for l in df['tokenized_Description']]
df.head()

,title,description,views,work_type,location,tokenized_Description,tokenized_desc_join
0,Licensed Insurance Agent,While many industries were hurt by the last fe...,5.0,FULL_TIME,"Chico, CA","[many, industries, hurt, last, people, still, ...",many industries hurt last people still need in...
1,Sales Manager,Are you a dynamic and creative marketing profe...,0.0,FULL_TIME,"Santa Clarita, CA","[dynamic, creative, marketing, professional, l...",dynamic creative marketing professional lookin...
2,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,17.0,CONTRACT,"New York, NY","[join, us, model, risk, auditor, showcase, fin...",join us model risk auditor showcase financial ...
3,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,0.0,FULL_TIME,"Forney, TX","[business, managerfirst, baptist, church, forn...",business managerfirst baptist church forneyfor...
4,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,2.0,FULL_TIME,"New York, NY","[could, one, magic, makers, ken, fulk, inc, se...",could one magic makers ken fulk inc seeking st...


In [15]:
model = Word2Vec(df['tokenized_Description'] ,
                               window=10,
                               min_count=10,
                               workers=5)
model.train(df['tokenized_Description'] ,total_examples=700,epochs=model.epochs)
model.build_vocab(df['tokenized_Description'],progress_per=100,update=True)

In [16]:
# data science specific skills
model.wv.most_similar(['statistics', 'machine', 'r', 'python',
                      'business', 'software', 'aws','sas','regression','predictive', 'modeling' ],topn=25)

[('visualization', 0.7985669374465942),
 ('query', 0.7753959894180298),
 ('algorithms', 0.7731486558914185),
 ('modelling', 0.7662094831466675),
 ('ml', 0.7605063915252686),
 ('etcexperience', 0.7498724460601807),
 ('etl', 0.7469797134399414),
 ('scripting', 0.7458682060241699),
 ('hadoop', 0.7401319146156311),
 ('statistical', 0.7385110855102539),
 ('powerbi', 0.7306546568870544),
 ('sql', 0.7275976538658142),
 ('nosql', 0.7239373922348022),
 ('matlab', 0.7238629460334778),
 ('nlp', 0.7228360176086426),
 ('ingestion', 0.7217429876327515),
 ('frameworks', 0.7193996906280518),
 ('apache', 0.7174987196922302),
 ('tableau', 0.7145667672157288),
 ('redshift', 0.7137537002563477),
 ('teradata', 0.7131528258323669),
 ('dax', 0.7105910181999207),
 ('github', 0.7080237865447998),
 ('looker', 0.7056930065155029),
 ('relational', 0.7014707326889038)]

In [17]:
temp = model.wv.most_similar(['finance', 'accounting', 'banking', 'business', 'management'],topn=10,)
temp

[('treasury', 0.7155026793479919),
 ('valuation', 0.6031790375709534),
 ('transactional', 0.5983290672302246),
 ('financial', 0.5844983458518982),
 ('fp', 0.5818897485733032),
 ('budgeting', 0.5602940917015076),
 ('traders', 0.5552447438240051),
 ('cfo', 0.5439025163650513),
 ('financials', 0.534822940826416),
 ('erp', 0.5297772884368896)]

In [18]:
temp[1][0]

'valuation'